In [5]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests as rq
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import RegexpTokenizer
import openpyxl
import spacy
import numpy as np
import time
import datetime
import os
import re
from tqdm import tqdm
from PIL import ImageTk, Image
from nltk import sent_tokenize
from wordcloud import WordCloud
from collections import Counter
from collections import defaultdict
# Load English language model
nlp = spacy.load("en_core_web_sm")
tokenizer = RegexpTokenizer(r'\w+')
# 加载预训练的语言模型
tokenizer = RegexpTokenizer(r'\w+')
#/////////////////////////////////////////////
nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')
font = 'msyh.ttc'  #設定字型
text_Set = []
# POS_Dict={}
POS_Dict_record = defaultdict(int)
# Dictionary for part of speech conversion
POS_Dict = {
    'Null': 'Null', 'cont': 'cont', 'CC': 'conj', 'CD': 'n', 'DT': 'art',
    'EX': 'pron', 'FW': 'fw', 'IN': 'prep', 'JJ': 'adj', 'JJR': 'adj',
    'JJS': 'adj', 'LS': 'n', 'MD': 'v', 'NN': 'n', 'NNS': 'n', 'NNP': 'n',
    'NNPS': 'n', 'PDT': 'n', 'POS': 'pron', 'PRP': 'pron', 'PRP$': 'pron',
    'RB': 'adv', 'RBR': 'adv', 'RBS': 'adv', 'RP': 'prep', 'SYM': 'sym',
    'TO': 'to', 'UH': 'n', 'VB': 'v', 'VBD': 'v', 'VBG': 'v',
    'VBN': 'v', 'VBP': 'v', 'VBZ': 'v', 'WDT': 'art', 'WP': 'pronoun',
    'WP$': 'pronoun', 'WRB': 'adv'
}

# SaveFile dictionary
SaveFile = {
    1: 'Eword', 2: 'Cword', 3: 'Eword_V', 4: 'Cword_V', 5: 'Eword_N',
    6: 'Cword_N', 7: 'Eword_AD', 8: 'Cword_AD'
}

# Function to get part of speech for tokens
# (She,pron)(is,verb)(a,art)(wonderful,adj)(cook,n)
def Get_Pos_dict2(tokens):
    poslist = []
    ANS_str = ""
#     pos_feats = keyin_pos_feats.value
    pos_feats = keyin_pos_feats.value.split(" ")
    Stopwords = Stop_word.value.split(" ")
#     print("pos_feats",pos_feats)
    poss = nltk.pos_tag(tokens)
    for pos in poss:
        try:
            ANS_str += f'{pos[0]}({POS_Dict[pos[1]]}),'
#             print(POS_Dict[pos[1]],pos[0])
            if pos[0] not in Stopwords:
                if "all" in pos_feats:
                    poslist.append(f'{pos[0]}')
    #                 print(pos[0],POS_Dict[pos[1]])
                    POS_Dict_record[POS_Dict[pos[1]]]+=1
                elif POS_Dict[pos[1]] in pos_feats:
                    poslist.append(f'{pos[0]}')
                    POS_Dict_record[POS_Dict[pos[1]]]+=1
    #                 print(pos[0],POS_Dict[pos[1]])
        except:
            pass
    return poslist
# Function to get content from Wikipedia
def Get_wiki_Context(url, headers_code=""):
    content_text = ""
    response = rq.get(url, headers=headers_code)
    bs = BeautifulSoup(response.text, 'html.parser')
    p_list = bs.find_all('p')
    for p in p_list:
        content_text += p.text.replace("\n", "")
    return content_text, "wiki"

# Function to process web URLs

def getWebText(web_url, filename):
    global urlfilename
    for i, url in enumerate(tqdm(web_url, desc="Processing URLs")):
        if "wiki" in url:
            context, filetype = Get_wiki_Context(url)
            urlfilename = url.split("/")[-1]
            # sentences = nltk.sent_tokenize(context)
            sentences = sent_tokenize(context) # 斷句
            text_POS = []
            text_NER = []
            for index, sentence in enumerate(sentences):
                tokens1 = tokenizer.tokenize(sentence)

                text_Set.extend(Get_Pos_dict2(tokens1))

    return context

# Function to handle button click
def on_submit_button_clicked(b):
    global urlfilename
    result = text_area.value
    text_Set.clear()
    POS_Dict_record.clear()
    web_urls = result.splitlines()
    getWebText(web_urls, category_name.value)
    if(sum(POS_Dict_record.values())<3):
        print("詞彙數=", sum(POS_Dict_record.values()),"詞彙數必須大於 3 !!")
        return
    print(POS_Dict_record)
    filename="propose_marriage.png"  #leaf.png  propose_marriage.jpg
    filename=text_image.value
    mask = np.array(Image.open(filename))
    diction = Counter(text_Set)
    wordcloud = WordCloud(background_color="white",mask=mask)  #背景顏色預設黑色,改為白色
    wordcloud.generate_from_frequencies(frequencies=diction)  #產生文字雲

    featfile=keyin_pos_feats.value.replace(" ","_")
    savename=f"Wordcloud_Result_{urlfilename}_{featfile}.png"
    print(savename)
    wordcloud.to_file(savename)  #存檔
    #     print()
    os.system("start mspaint.EXE " + savename)
    show_word.value = "Processing complete."

# Text widgets
text_area = widgets.Text(
    value='',
    placeholder='Enter URL',
    description='輸入網址:',
    layout=widgets.Layout(width='80%')
)
text_image = widgets.Text(
    value='propose_marriage.png',
    placeholder='propose_marriage.png',
    description='image_file',
    layout=widgets.Layout(width='80%')
)

category_name = widgets.Text(
    value='Cut_POS_NER',
    description='Category:',
    layout=widgets.Layout(width='80%')
)

Stop_word = widgets.Text(
    value='be is are',
    placeholder='Stop Word:',
    description='輸入停用詞',
    layout=widgets.Layout(width='80%')
)

keyin_pos_feats = widgets.Text(
    value='all',
    placeholder='輸入POS or all:',
    description='輸入POS',
    layout=widgets.Layout(width='80%')
)

submit_button = widgets.Button(
    description='確定',
    button_style='success',
    tooltip='Click to submit'
)
urlfilename=""
# pos_feats=["verb","n","adj","all"]
# Display widgets
display( category_name,text_image,Stop_word,keyin_pos_feats, text_area, submit_button)

# Output area
show_word = widgets.Label(value="Waiting..")
display(show_word)

# Bind button clicks to functions
submit_button.on_click(on_submit_button_clicked)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Text(value='Cut_POS_NER', description='Category:', layout=Layout(width='80%'))

Text(value='propose_marriage.png', description='image_file', layout=Layout(width='80%'), placeholder='propose_…

Text(value='be is are', description='輸入停用詞', layout=Layout(width='80%'), placeholder='Stop Word:')

Text(value='all', description='輸入POS', layout=Layout(width='80%'), placeholder='輸入POS or all:')

...

defaultdict(<class 'int'>, {'n': 2869, 'prep': 704, 'art': 629, 'v': 943, 'adj': 438, 'adv': 191, 'conj': 225, 'to': 128, 'pron': 88, 'fw': 1, 'pronoun': 7})
Wordcloud_Result_IPhone_all.png


Processing URLs: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.47s/it]


defaultdict(<class 'int'>, {'v': 943})
Wordcloud_Result_IPhone_v.png


Processing URLs: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.55s/it]


defaultdict(<class 'int'>, {'v': 871})
Wordcloud_Result_IPhone_v.png
